In [64]:
import pandas as pd
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean

In [65]:
read_final = True

if read_final:
    product = pd.read_pickle('./datathon/dataset/final_products')
    outfits = pd.read_pickle('./datathon/dataset/final_outfits')
else:
    o_outfits= pd.read_csv('./datathon/dataset/outfit_data.csv')
    o_product = pd.read_csv('./datathon/dataset/product_data.csv')

    outfits = o_outfits.copy()
    product = o_product.copy()

    #Deleting all the products that are not clothes
    product = product[(product.des_product_category!= 'Beauty') & (product.des_product_category!= 'Home')]
    # Delete all products for men and kids due to desbalance
    product = product[(product.des_line== 'SHE')]
    product = product[(product.des_sex== 'Female')]
    product = product[(product.des_age== 'Adult')]
    product = product[~product.des_product_aggregated_family.isin(['Dresses and jumpsuits','Swim and intimate','Accessories'])]
    product['des_product_category'] = product['des_product_category'].apply(lambda x: 'Tops' if x =='Outerwear' else x)
    product =product.drop(['des_sex','des_line','des_age'], axis = 1)

    # Delete the deleted products from outfits
    outfits = outfits[outfits.cod_modelo_color.isin(product.cod_modelo_color.unique())]

In [68]:
product.head()

,cod_modelo_color,cod_color_code,des_color_specification_esp,des_agrup_color_eng,des_fabric,des_product_category,des_product_aggregated_family,des_product_family,des_product_type,des_filename,emb
0,41085800-02,02,OFFWHITE,WHITE,P-PLANA,Bottoms,Trousers & leggings,Trousers,Trousers,datathon/images/2019_41085800_02.jpg,"[1.6967614, 1.6967614, 1.6967614, 1.6967614, 1..."
1,53000586-TO,TO,TEJANO OSCURO,BLUE,J-JEANS,Bottoms,Jeans,Jeans,Jeans,datathon/images/2019_53000586_TO.jpg,"[1.6821629, 1.6967614, 1.6967614, 1.6967614, 1..."
4,53070773-70,70,ROJO,RED,P-PLANA,Tops,Shirts,Shirt,Shirt,datathon/images/2019_53070773_70.jpg,"[1.6383677, 1.6383677, 1.6383677, 1.6383677, 1..."
5,53003736-94,94,GRIS MEDIO VIGORE,GREY,T-TRICOT,Tops,Sweaters and Cardigans,Sweater,Sweater,datathon/images/2019_53003736_94.jpg,"[1.5799739, 1.5799739, 1.5653756, 1.5653756, 1..."
6,53045718-17,17,CARAMELO,YELLOW,T-TRICOT,Tops,Sweaters and Cardigans,Sweater,Sweater,datathon/images/2019_53045718_17.jpg,"[1.5507771, 1.5653756, 1.5799739, 1.5799739, 1..."


## AUX FUNCTIONS 

In [69]:
def getProductIdx(idx):
    print("DATOS DEL PRODUCTO")
    print(product.iloc[idx])
    print('OUTFITS APPEARING IN:')
    code = product.iloc[idx].cod_modelo_color
    print(list(outfits[outfits.cod_modelo_color==code]['cod_outfit'].unique()))
    print('IMAGE')
    image_path =product.loc[idx]['des_filename']
    img = cv2.imread(image_path)
    plt.imshow(img)


In [70]:
def getProductByCode(code):
    print("DATOS DEL PRODUCTO")
    print(product.loc[product.cod_modelo_color == code])
    print('OUTFITS APPEARING IN:')
    print(list(outfits[outfits.cod_modelo_color==code]['cod_outfit'].unique()))
    print('IMAGE')
    image_path =product[product.cod_modelo_color == code]['des_filename'].values[0]
    img = cv2.imread(image_path)
    plt.imshow(img)


In [71]:
def printOufit(outfit):
    listaPrednas =  outfits[outfits.cod_outfit==1]['cod_modelo_color'].unique()
    for i in listaPrednas:
        image_path =product[product.cod_modelo_color == i]['des_filename'].values[0]
        img = cv2.imread(image_path)
        plt.imshow(img)
        plt.show()

In [72]:
def getEmbeding(code):
    if code == None:
        return None
    return product[product.cod_modelo_color==code]['emb'].values

In [73]:
# Calulate probabilities
outfits['col'] = outfits['cod_modelo_color'].apply(lambda x: x.split('-')[1])
df_grouped = outfits.groupby('cod_outfit')['col'].apply(list).reset_index()

conteo_color = {}
for i in product["cod_color_code"].unique():
    conteo_color[i] = 0
    for j in df_grouped.iloc[:,1].values:
        for k in j:
            if k == i:
                for z in range(0,len(j)):
                    conteo_color[k+j[z]] = 0

for i in product["cod_color_code"].unique():
    conteo_color[i] = 0
    for j in df_grouped.iloc[:,1].values:
        for k in j:
            if k == i:
                conteo_color[i] += 1
                for z in range(0,len(j)):
                    if k != j[z]:
                        conteo_color[k+j[z]] += 1
                    else:
                        conteo_color[k+j[z]] += .5

In [74]:
# Probabilidad condiciional de code2 once you have code 1
def probabilidad_col(colorcode1, colorcode2):
    code = colorcode1 + colorcode2
    prob = conteo_color[code]/conteo_color[colorcode1]
    return prob

In [75]:
outfits['fab'] = outfits["cod_modelo_color"].apply(lambda x: product[(product['cod_modelo_color'] == x)].iloc[0][7])
df_grouped_fab = outfits.groupby('cod_outfit')['fab'].apply(list).reset_index()

tejido = {}
for i in product["des_fabric"].unique():
    tejido[i] = 0
    for j in df_grouped_fab.iloc[:,1].values:
        for k in j:
            if k == i:
                for z in range(0,len(j)):
                    tejido[k+j[z]] = 0

for i in product["des_fabric"].unique():
    tejido[i] = 0
    for j in df_grouped_fab.iloc[:,1].values:
        for k in j:
            if k == i:
                tejido[i] += 1 * len(j)
                for z in range(0,len(j)):
                    if k != j[z]:
                        tejido[k+j[z]] += 1
                    else:
                        tejido[k+j[z]] += .5

In [76]:
def probabilidad_fab(fabric1, fabric2):
    fabric = fabric1 + fabric2
    try:
        prob = tejido[fabric]/tejido[fabric1]
        return prob
    except:
        return 0

# BUSQUEDA LOCAL

In [77]:
def computeHeuristicPorbs(codes):
    
    fab1 = product[product.cod_modelo_color==codes[0]]['des_fabric'].values[0]
    fab2 = product[product.cod_modelo_color==codes[1]]['des_fabric'].values[0]
    
    col1 = product[product.cod_modelo_color==codes[0]]['cod_color_code'].values[0]
    col2 = product[product.cod_modelo_color==codes[1]]['cod_color_code'].values[0]
    a=0
    b=0
    c=0
    d=0
    if len(codes) ==3:
        fab3 = product[product.cod_modelo_color==codes[2]]['des_fabric'].values[0]
        col3 = product[product.cod_modelo_color==codes[2]]['cod_color_code'].values[0]
        b=probabilidad_fab(fab1,fab3)
        d=probabilidad_col(col1,col3)
        
    a=probabilidad_fab(fab1,fab2)   
    c=probabilidad_col(col1,col2)
    
    mean= (a+b+c+d) /len(codes)
    return mean

In [78]:
computeHeuristicPorbs(['51000622-02','43067759-01'])

0.14753587024329382

In [98]:
def computeHeuristic(embedings):
    sum_distances=0
    for i in range(len(embedings)):
        for j in range(i + 1, len(embedings)):
            distance_ij = np.linalg.norm(embedings[i][0] - embedings[j][0])
            sum_distances += distance_ij
    return sum_distances

In [105]:
def getNewProdSucc(estado, part):    
    data = product.copy()
    match part:
        case "top":
            data  = product.loc[product['des_product_category'] == "Tops"]
        case "bot":
            data = product.loc[product['des_product_category'] == "Bottoms"]
        case "shoes":
            data  = product.loc[product['des_product_family'] == "Footwear"]
    newState = None
    while(newState == None):
        randomObs = data.sample(1)
        newProd = randomObs.cod_modelo_color.values[0]
        newState = None
        
        match part:
            case "top":
                 newState = State(newProd, estado.get_bot(), estado.get_shoes(),depth= (estado.get_depth()+1))
            case "bot":
                 newState = State(estado.get_top(), newProd, estado.get_shoes(),depth=(estado.get_depth()+1))
            case "shoes":
                 newState = State(estado.get_top(), estado.get_bot(), newProd,depth=(estado.get_depth()+1))
            
        newStateH = newState.get_heuristica()
        if newStateH < estado.get_heuristica():
            return newProd 
        newState, newStateH = None, None # reset

In [100]:
MAX_DEPTH = 100

In [120]:
def hillClimbing(estado):
    successors = []
    random_seed = None
    
    # top sucesores
    topPart = None
    while (topPart == None):
        topPart = getNewProdSucc(estado, "top")
        newState = State(topPart, estado.get_bot(), estado.get_shoes(),depth=(estado.get_depth()+1))
        
        newStateH = newState.get_heuristica()
        if(newStateH <= estado.get_heuristica()):
            newState, newStateH = None, None
        else:
            successors.append((newState, newState.get_heuristica()))
        
    
    # bottom sucesores
    bottomPart = None
    while(bottomPart == None):
        bottomPart = getNewProdSucc(estado, "bot")
        newState = State(estado.get_top(), bottomPart, estado.get_shoes(),depth=(estado.get_depth()+1))
        
        newStateH = newState.get_heuristica()
        if(newStateH <= estado.get_heuristica()):
            newState, newStateH = None, None
        else:
            successors.append((newState, newState.get_heuristica()))
        
    # zapatos sucesores
    shoesPart = None
    while (shoesPart == None):
        shoesPart = getNewProdSucc(estado, "shoes")
        newState = State(estado.get_top(), estado.get_bot(), shoesPart, depth=(estado.get_depth()+1))
        
        newStateH = newState.get_heuristica()
        if(newStateH <= estado.get_heuristica()):
            newState, newStateH = None, None
        else:
            successors.append((newState, newState.get_heuristica()))
        
    
    if (len(successors) == 0 or (estado.get_depth()+1) > MAX_DEPTH): return estado
    
    # get first as best
    maxH = successors[0].item(0)
    bestSuccessor = successors[0].item(0)
    
    for succ in successors:
        if(maxH < succ.item(1)):
            maxH = succ.item(1)
            bestSuccessor = succ.item(0)
    print(bestSuccessor.get_depth())
    
    return hillClimbing(bestSuccessor)

In [117]:
class State:
    def __init__(self, top=None, bot=None, shoes=None, heuristica=np.inf, depth = 0):
        self.top = top
        self.bot = bot
        self.shoes = shoes
        self.heuristica = heuristica
        self._depth = depth
        if (self.top!= None or self.shoes!=None or self.bot!=None): self.heuristica = computeHeuristic(self.getAllEmbedings())
        print('DEPTH',self.get_depth())
        print('H',self.get_heuristica())
        
    def getAllEmbedings(self):
        embedings = [getEmbeding(self.top),getEmbeding(self.bot),getEmbeding(self.shoes)]
        embedings = [emb for emb in embedings if emb!=None]
        return embedings

        
    def getProducts(self):
        fig, axs = plt.subplots(1, 3, figsize=(10, 3))  # Create a 1x3 grid of subplots

        # Top
        top_image_path = product[product.cod_modelo_color == self.top]['des_filename'].values[0]
        top_img = cv2.imread(top_image_path)
        axs[0].imshow(top_img)
        axs[0].set_title('Top')

        # Bottom
        bot_image_path = product[product.cod_modelo_color == self.bot]['des_filename'].values[0]
        bot_img = cv2.imread(bot_image_path)
        axs[1].imshow(bot_img)
        axs[1].set_title('Bottom')

        # Shoes
        shoes_image_path = product[product.cod_modelo_color == self.zapatos]['des_filename'].values[0]
        shoes_img = cv2.imread(shoes_image_path)
        axs[2].imshow(shoes_img)
        axs[2].set_title('Shoes')

        plt.show()
    def show_top(self):
        return getProductByCode(self.top)

    def show_bot(self):
        return getProductByCode(self.bot)

    def show_shoes(self):
        return getProductByCode(self.shoes)
    
    def get_top(self):
        return self.top

    def get_bot(self):
        return self.bot

    def get_shoes(self):
        return self.shoes
    
    def get_depth(self):
        return self._depth
    
    def get_heuristica(self):
        return self.heuristica

In [ ]:
outfit0= State(top='51000622-02',bot='43067759-01')

In [121]:
out_fin= hillClimbing(outfit0)

C:\Users\lluis\AppData\Local\Temp\ipykernel_12900\3830255375.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  embedings = [emb for emb in embedings if emb!=None]


DEPTH 1
H 145.9803924560547
DEPTH 1
H 643.3787231445312
DEPTH 1
H 726.8472290039062
DEPTH 1
H 726.5983276367188
DEPTH 1
H 79.71943664550781
DEPTH 1
H 80.39429473876953
DEPTH 1
H 231.87396240234375
DEPTH 1
H 592.673583984375
DEPTH 1
H 89.04190826416016
DEPTH 1
H 97.912841796875
DEPTH 1
H 60.197025299072266
DEPTH 1
H 390.31463623046875
DEPTH 1
H 119.89968872070312
DEPTH 1
H 455.46514892578125
DEPTH 1
H 381.256591796875
DEPTH 1
H 68.68055725097656
DEPTH 1
H 319.98260498046875
DEPTH 1
H 788.4625854492188
DEPTH 1
H 399.5298767089844
DEPTH 1
H 276.3306884765625
DEPTH 1
H 268.66070556640625
DEPTH 1
H 95.19758605957031
DEPTH 1
H 220.88833618164062
DEPTH 1
H 523.9595336914062
DEPTH 1
H 78.0718765258789
DEPTH 1
H 91.1141128540039
DEPTH 1
H 486.1481628417969
DEPTH 1
H 152.17001342773438
DEPTH 1
H 714.0802001953125
DEPTH 1
H 573.7322998046875
DEPTH 1
H 837.7484130859375
DEPTH 1
H 33.44797134399414
DEPTH 1
H 33.44797134399414
DEPTH 1
H 853.346923828125
DEPTH 1
H 738.3116455078125
DEPTH 1
H 351.2630

DEPTH 1
H 834.0379867553711
DEPTH 1
H 906.833122253418
DEPTH 1
H 296.4126281738281
DEPTH 1
H 351.99629974365234
DEPTH 1
H 346.2364730834961
DEPTH 1
H 349.3618850708008
DEPTH 1
H 316.35843658447266
DEPTH 1
H 305.8665542602539
DEPTH 1
H 642.5210189819336
DEPTH 1
H 363.7147903442383
DEPTH 1
H 913.5205917358398
DEPTH 1
H 477.55953216552734
DEPTH 1
H 185.10559844970703
DEPTH 1
H 728.9049911499023
DEPTH 1
H 1110.8395309448242
DEPTH 1
H 455.90760040283203
DEPTH 1
H 462.2577438354492
DEPTH 1
H 550.8925857543945
DEPTH 1
H 480.68933868408203
DEPTH 1
H 402.48717498779297
DEPTH 1
H 303.0421829223633
DEPTH 1
H 648.4669418334961
DEPTH 1
H 319.3401107788086
DEPTH 1
H 451.02938079833984
DEPTH 1
H 251.37864685058594
DEPTH 1
H 470.20801544189453
DEPTH 1
H 662.6746139526367
DEPTH 1
H 375.6684799194336
DEPTH 1
H 428.36376190185547
DEPTH 1
H 377.26317596435547
DEPTH 1
H 201.72930908203125
DEPTH 1
H 193.0783462524414
DEPTH 1
H 454.8661117553711
DEPTH 1
H 319.3401107788086
DEPTH 1
H 273.6819534301758
DEPTH 1

DEPTH 1
H 902.8255233764648
DEPTH 1
H 690.604606628418
DEPTH 1
H 358.1368637084961
DEPTH 1
H 525.007438659668
DEPTH 1
H 868.6067428588867
DEPTH 1
H 901.3849105834961
DEPTH 1
H 223.2481231689453
DEPTH 1
H 844.9714584350586
DEPTH 1
H 280.38350677490234
DEPTH 1
H 291.88162994384766
DEPTH 1
H 452.6055374145508
DEPTH 1
H 405.0783004760742
DEPTH 1
H 1033.8537521362305
DEPTH 1
H 316.35843658447266
DEPTH 1
H 361.07662200927734
DEPTH 1
H 1122.241569519043
DEPTH 1
H 237.83441162109375
DEPTH 1
H 251.37864685058594
DEPTH 1
H 814.2767562866211
DEPTH 1
H 352.04317474365234
DEPTH 1
H 402.48717498779297
DEPTH 1
H 815.1213607788086
DEPTH 1
H 457.3651657104492
DEPTH 1
H 251.37864685058594
DEPTH 1
H 473.6937942504883
DEPTH 1
H 287.3357620239258
DEPTH 1
H 243.98849487304688
DEPTH 1
H 482.8797073364258
DEPTH 1
H 627.4056015014648
DEPTH 1
H 227.48199462890625
DEPTH 1
H 449.8446578979492
DEPTH 1
H 633.6846542358398
DEPTH 1
H 429.0073013305664
DEPTH 1
H 284.72740936279297
DEPTH 1
H 571.677848815918
DEPTH 1
H 

KeyboardInterrupt: 

In [ ]:
out_fin

In [122]:
print("prd")

prd
